In [1]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [2]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
from os import environ
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [3]:
root = "/data/gpfs/assoc/denovo/tfaske/frogs"

In [4]:
cd $root

/data/gpfs/assoc/denovo/tfaske/frogs


In [5]:
pwd

'/data/gpfs/assoc/denovo/tfaske/frogs'

## Assembly

with dDocent c=.92, k1=6, k2=6

In [6]:
assembly = "/data/gpfs/assoc/denovo/tfaske/frogs/assembly/reference.fasta"

In [8]:
!samtools faidx $assembly

## Actual Mapping 
 

In [12]:
fastq_files = []
files = !find newFastq -name '*fq.gz'
files = [os.path.abspath(x) for x in files]
for x in files:
    fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [13]:
len(fastq_files),fastq_files[0]

(323, '/data/gpfs/assoc/denovo/tfaske/frogs/newFastq/RL_AD1C_200.F.fq.gz')

In [14]:
cd $root

/data/gpfs/assoc/denovo/tfaske/frogs


In [15]:
# -k INT minimum seed length [19]
# -w INT band width for banded alignment [100]
# -r FLOAT look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
# -T INT minimum score to output [30]
# -R STR read group header line such as '@RG\tID:foo\tSM:bar' [null]

#@lview.remote()
def run_bwamem(args):
    import os, multiprocessing, socket
    cpus = 1
    assembly, fq, outdir = args
    ID = fq.split('/')[8] ### need to change this to match your ID 
    ID = ID.split('.F.')[0] ### This too 
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(ID)))
    bam = sam.replace('.sam','.bam')
    bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
    bwa_cmd = r"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\tID:%s\tLB:%s\tSM:%s\tPL:ILLUMINA' %s %s > %s" % (ID,ID,ID,assembly,fq,sam)
    s2b_cmd =  "samtools view -b %s -o %s\n\nsamtools sort -@ %s %s -o %s\n\nsamtools index %s\n\n" % (sam,bam,cpus,bam,bam_sorted,bam_sorted)                                                              
    return  bwa_cmd,s2b_cmd 

In [16]:
!mkdir SNPcall

In [17]:
!mkdir SNPcall/bwa

In [18]:
!mkdir SNPcall/bwa/shdir

In [19]:
bwa_dir = os.path.join(root,"SNPcall/bwa/")
assert(bwa_dir)

In [20]:
### creates a list of commands for bwa-mem for each fastq file
res_bwa = []
res_s2b = []
for f in fastq_files:
    r1,r2 = run_bwamem((assembly, f, bwa_dir))
    res_bwa.append(r1)
    res_s2b.append(r2)

In [21]:
len(res_bwa),res_bwa[0]

(323,
 "bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:RL_AD1C_200\\tLB:RL_AD1C_200\\tSM:RL_AD1C_200\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/frogs/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/frogs/newFastq/RL_AD1C_200.F.fq.gz > /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200.sam")

In [22]:
len(res_s2b),res_s2b[0]

(323,
 'samtools view -b /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200.sam -o /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200.bam\n\nsamtools sort -@ 1 /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200.bam -o /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200_sorted.bam\n\nsamtools index /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200_sorted.bam\n\n')

In [23]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa


#### Below selects options for slurm submission and is a function for creating a slurm script per fastq

In [24]:
fq_ID = [fq.split('/')[8].split('.F.')[0] for fq in fastq_files]

In [25]:
len(fq_ID), fq_ID[0]

(323, 'RL_AD1C_200')

In [26]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'bwa_RALU'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 10000
email = 'tfaske@nevada.unr.edu'

In [27]:
def write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,bwa_cmds,s2b_cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(bwa_cmds)):
        with open("shdir/run_bwamem_%s.sh" % (fq_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bwamem
#SBATCH --output bwa/shdir/output_bwamem_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n\n
    
%s \n\n
%s \n""" % (account,partition,time,int(cpus),int(mem_cpu),fq_ID[i],fq_ID[i],email,bwa_cmds[i],s2b_cmds[i]))

In [28]:
write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,res_bwa,res_s2b)

#### finds all bwa slurm scripts and writes bash script to sbatch them

In [29]:
shbwa_files = []
files = !find ./shdir -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [30]:
len(shbwa_files),shbwa_files[0]

(323,
 '/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/shdir/run_bwamem_RL_AD1C_200.sh')

In [31]:
def write_bash_bwamem_sh(sh_files):
    with open("SNPcall/run_bash_bwa.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [32]:
cd $root

/data/gpfs/assoc/denovo/tfaske/frogs


In [33]:
write_bash_bwamem_sh(shbwa_files)

# Run run_bash_bwamem_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/frogs/SNPcall 
    source activate py36
    bash run_bash_bwa.sh
    
## Remove sam files and unsorted bams

       whatever, run in terminal. Move sorted bams, rm bam, mv back

# Calculates coverage from bam_files (do later, after SNP call) 

need to rewrite to calculate individually, save data, and write mean to outfile. 

also outputs a file called "bam_coverage.csv" in the bam folder

samtools depth -a AT2_BB_10_mapped.bam  |  awk '{sum+=$3} END { print "Average = ",sum/NR}'

In [34]:
samtools = "samtools"

In [35]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa


In [37]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

len(bam_files), len(fastq_files)

(323, 323)

In [38]:
len(bam_files), bam_files[0]

(323,
 '/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200_sorted.bam')

In [39]:
bam_names = []
cov_list = []
for i in range(0,len(bam_files)):
    bam = bam_files[i]
    print(bam)
    b = bam.split('/')[9] #set this 
    #print (b)
    !$samtools depth -a $b > $'cov.txt'
    cov = pd.read_csv('cov.txt', sep="\t",header=None)
    coverage = sum(cov.iloc[:,2])/len(cov.index)
    name = b.split('.F')[0]
    #print(name)
    bam_names.append(name)
    cov_list.append(coverage)
    print(coverage,name)
cov_df = pd.DataFrame(bam=bam_names,coverage=cov_list)
cov_df.head()

/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_200_sorted.bam
11.865526135744542 RL_AD1C_200_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_201_sorted.bam
17.385402250919356 RL_AD1C_201_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD1C_202_sorted.bam
21.299097084551757 RL_AD1C_202_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD2C_203_sorted.bam
17.288640772940482 RL_AD2C_203_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD2C_204_sorted.bam
20.830907220081848 RL_AD2C_204_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD2C_205_sorted.bam
19.87799407951039 RL_AD2C_205_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD2C_206_sorted.bam
19.68170329198031 RL_AD2C_206_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD2C_210_sorted.bam
16.480141224347772 RL_AD2C_210_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_AD2C_211_sorted.bam
15.931082501444141

1.095601552393273 RL_FPC_3_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4314_sorted.bam
8.543735294763632 RL_FPH_4314_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4317_sorted.bam
4.265762826604871 RL_FPH_4317_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4320_sorted.bam
6.52775010858131 RL_FPH_4320_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4338_sorted.bam
9.373571228626274 RL_FPH_4338_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4340_sorted.bam
7.178011151216654 RL_FPH_4340_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4341_sorted.bam
8.367532362858402 RL_FPH_4341_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4342_sorted.bam
4.874152903450158 RL_FPH_4342_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_4368_sorted.bam
11.47587031012803 RL_FPH_4368_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_FPH_437

15.646744613459509 RL_PBH_473_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PBH_478_sorted.bam
1.4774994433311066 RL_PBH_478_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PBH_483_sorted.bam
11.178413552977867 RL_PBH_483_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PBH_531_sorted.bam
14.509880449791234 RL_PBH_531_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PBH_533_sorted.bam
11.998388945715742 RL_PBH_533_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PBH_540_sorted.bam
9.072237404471688 RL_PBH_540_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PCH_2025_sorted.bam
11.1476347002722 RL_PCH_2025_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PCH_2028_sorted.bam
6.328940717813553 RL_PCH_2028_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PCH_562_sorted.bam
14.608921945647072 RL_PCH_562_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_PCH_566_sor

18.935834981951196 RL_S8C_97_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_S8C_98_sorted.bam
15.941874342449271 RL_S8C_98_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_S8C_99_sorted.bam
15.827942227902717 RL_S8C_99_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_220_sorted.bam
15.82788800157215 RL_UBPC_220_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_230_sorted.bam
17.814519627549547 RL_UBPC_230_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_231_sorted.bam
16.830049775529318 RL_UBPC_231_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_232_sorted.bam
19.69496450033729 RL_UBPC_232_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_233_sorted.bam
15.5349947556461 RL_UBPC_233_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_234_sorted.bam
21.730873782028205 RL_UBPC_234_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_UBPC_240

12.202738042888454 RL_WRH_2441_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_2443_sorted.bam
12.285577012861568 RL_WRH_2443_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_2448_sorted.bam
16.25294753772638 RL_WRH_2448_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_2466_sorted.bam
11.54144383092116 RL_WRH_2466_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_267_sorted.bam
8.76094771285763 RL_WRH_267_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_270_sorted.bam
12.087946789768727 RL_WRH_270_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_273_sorted.bam
1.6921851051971575 RL_WRH_273_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_276_sorted.bam
12.26330181670892 RL_WRH_276_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_284_sorted.bam
2.1232195370640934 RL_WRH_284_sorted.bam
/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa/RL_WRH_709_s

TypeError: __init__() got an unexpected keyword argument 'bam'

In [40]:
cov_dict = {"bam":bam_names,'coverage':cov_list}
cov_df = pd.DataFrame(cov_dict)
cov_df.head()
#len(cov_df)

,bam,coverage
0,RL_AD1C_200_sorted.bam,11.865526
1,RL_AD1C_201_sorted.bam,17.385402
2,RL_AD1C_202_sorted.bam,21.299097
3,RL_AD2C_203_sorted.bam,17.288641
4,RL_AD2C_204_sorted.bam,20.830907


In [41]:
cov_out = os.path.join(bwa_dir,'bam_coverage.csv')
cov_df.to_csv(path_or_buf=cov_out)

In [42]:
cov_df.coverage.describe()


count    323.000000
mean      14.828440
std        5.384907
min        1.095602
25%       12.065928
50%       16.124611
75%       18.806876
max       27.724190
Name: coverage, dtype: float64

In [43]:
len(cov_df.coverage)

323

# move good bams

In [46]:
len(cov_df[cov_df.coverage > 10]),len(cov_df[cov_df.coverage >= 7.5]),len(cov_df[cov_df.coverage >= 5])

(267, 284, 297)

In [50]:
good_bam = cov_df.bam[cov_df.coverage >= 5]
good_bam = good_bam.tolist()
len(good_bam),good_bam[0]

(297, 'RL_AD1C_200_sorted.bam')

In [51]:
good_bam_files = []
for i in range(0,len(good_bam)):
    bam = bwa_dir + '/' + good_bam[i]
    good_bam_files.append(bam)
len(good_bam_files), good_bam_files[1]

(297,
 '/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa//RL_AD1C_201_sorted.bam')

In [52]:
snp_dir = os.path.join(root,'SNPcall')

In [53]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall


In [54]:
!mkdir 'good_bams'

In [55]:
good_bam_dir = os.path.join(snp_dir,'good_bams')
assert good_bam_dir

In [56]:
for i in range(0,len(good_bam_files)):
    good_bam = str(good_bam_files[i])
    !cp $good_bam $good_bam_dir

In [57]:
cd $good_bam_dir

/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/good_bams


In [58]:
good_bam_cp_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    good_bam_cp_files.append(x)
good_bam_cp_files = sorted(good_bam_files)

In [59]:
len(good_bam_cp_files),good_bam_cp_files[0]

(297,
 '/data/gpfs/assoc/denovo/tfaske/frogs/SNPcall/bwa//RL_AD1C_200_sorted.bam')